# Retrieve data from eNanomapper database
https://search.data.enanomapper.net/

- This notebook uses Apache Solr API and AMBIT REST API  (see Swagger interactive documentation at http://enanomapper.github.io/API/ )

In [2]:
import pandas as pd
import os.path
import numpy as np
import datetime, time
import urllib3
import json
import sys
import ipywidgets as widgets
import logging
from logging.config import fileConfig
fileConfig('logging_endpoints_config.ini')

from jaqpotpy import Jaqpot

global logger
logger = logging.getLogger()

%run units.py

logger.debug('Started at %s \t%s',os.name, datetime.datetime.now())

urllib3.disable_warnings()

In [3]:
%run solrapi.py

In [4]:
%run enmapi.py

In [5]:
http_pool=urllib3.PoolManager()

### Retrieve endpoints using AMBIT REST API

In [6]:
ambit=AMBITQuery(key="study")
print(ambit.root)
data = ambit.get(http_pool)


2018-12-08 00:26:41,454  INFO     Sending query to https://apps.ideaconsult.net/nanoreg1/query/study


https://apps.ideaconsult.net/nanoreg1


2018-12-08 00:26:41,879  INFO     Received response 


In [7]:
#Select endpoint
_sections={}

top_sections = []
for facet in data['facet']:
    #print("{}\t{}\t{}\t{}".format(facet['subcategory'],facet['endpoint'],facet['count'],facet['value']))
    top = facet['subcategory']
    if not top in top_sections:
        top_sections.append(top)
        
    if not top in _sections:
        _sections[top] = []
    
    _sections[top].append(facet['endpoint'])

top_widget = widgets.Dropdown(
    options=top_sections,
    value=top_sections[0],
    description='Select:',
    disabled=False,
)
display(top_widget)


Dropdown(description='Select:', options=('ECOTOX', 'P-CHEM', 'TOX'), value='ECOTOX')

In [8]:
top=top_widget.value
category_widget = widgets.Dropdown(
    options=_sections[top],
    value=_sections[top][1],
    description='Endpoint:',
    disabled=False,
)
display(category_widget)

Dropdown(description='Endpoint:', index=1, options=('EC_ALGAETOX_SECTION', 'EC_DAPHNIATOX_SECTION', 'EC_SOILDW…

#### Setup the query

In [9]:
freetext_widget=widgets.Text(
    value='NM220,NM101',
    description='Search',
    disabled=False
)
freetext_widget

Text(value='NM220,NM101', description='Search')

In [10]:
materialfilter=freetext_widget.value
if "" == materialfilter:
    materialfilter=None
    
section=category_widget.value
fields=None
if section in _fields:
    fields=_fields[section]

endpoint="*"
if section in _endpoint:
    endpoint=_endpoint[section]
    
settings = {'endpointfilter': ' effectendpoint_s: {}'.format(endpoint),
            'studyfilter': ' topcategory_s:({}) AND endpointcategory_s:({})'.format(top,section),            
            'query_guidance': None, 
            'query_organism': None,
            'fields' : fields
           }
    
logger.info(settings)  
logger.info("Free text query: {}".format(materialfilter))

2018-12-08 00:26:44,802  INFO     {'endpointfilter': ' effectendpoint_s: *', 'studyfilter': ' topcategory_s:(ECOTOX) AND endpointcategory_s:(EC_DAPHNIATOX_SECTION)', 'query_guidance': None, 'query_organism': None, 'fields': None}
2018-12-08 00:26:44,804  INFO     Free text query: NM220,NM101


### Run the query

In [11]:

try:
    with open('./config.solr.json') as f:
        solr_services = json.load(f)
except:        
    solr_services = None
    logger.error("Something went wrong")

service_widget = widgets.Dropdown(
    options=solr_services.keys(),
    description='Service:',
    disabled=False,
)
display(service_widget)

name_widget=widgets.Text(
    placeholder='',
    description='User name:',
    disabled=False
)
password_widget = widgets.Password(description='Password:', placeholder='')
display(name_widget)
password_widget

Dropdown(description='Service:', options=('nanoreg1', 'enm'), value='nanoreg1')

Text(value='', description='User name:', placeholder='')

Password(description='Password:', placeholder='')

In [12]:
query=getSolrQuery(settings,url=solr_services[service_widget.value])
rows = []  

#logger.info(settings)
logger.info(query)
rows = sendSolrRequest(settings=settings,http=http_pool,textfilter=materialfilter,query=query,rows=rows,basic_auth="{}:{}".format(name_widget.value,password_widget.value))

2018-12-08 00:26:46,632  INFO     {'url': 'https://solr.ideaconsult.net/solr/nanoreg_shard1_replica1/select', 'fl': 'name_hs,publicname_hs,substanceType_hs,s_uuid_hs,[child parentFilter=filter(type_s:substance) childFilter="filter(type_s:study AND  topcategory_s:(ECOTOX) AND endpointcategory_s:(EC_DAPHNIATOX_SECTION) AND  effectendpoint_s: *)  OR filter(type_s:params AND  topcategory_s:(ECOTOX) AND endpointcategory_s:(EC_DAPHNIATOX_SECTION))  OR filter(type_s:conditions AND  topcategory_s:(ECOTOX) AND endpointcategory_s:(EC_DAPHNIATOX_SECTION)) OR filter(type_s:composition AND component_s:CONSTITUENT)" limit=10000]', 'fq': '', 'q': '{!parent which=type_s:substance}'}
2018-12-08 00:26:46,634  INFO     Sending query to https://solr.ideaconsult.net/solr/nanoreg_shard1_replica1/select
2018-12-08 00:26:47,214  INFO     {
  "name_hs": "NM-100 (TiO2 50-150 nm)",
  "publicname_hs": "JRCNM01000a",
  "substanceType_hs": "NPO_1486",
  "s_uuid_hs": "NNRG-18280a4a-45e9-adc0-df3b-125397b1255f"
}


In [13]:
#print("Substances: {}".format(len(rows)))
df = pd.DataFrame(rows)
df.to_csv(section+".nosmiles.txt",sep='\t',index=False)
#df.head()
df.head()

,public.name,r.purposeFlag,r.reliability,r.studyResultType,substance.name,substance.type,substance.uuid,value.endpoint,value.lo,value.qualifier.lo,...,x.params.MEDIUM.CO2_concentration_%v/v,x.params.MEDIUM.O2_concentration_%v/v,x.params.MEDIUM.composition,x.params.MEDIUM.ionictrength_m,x.params.MEDIUM.ionictrength_mM,x.params.MEDIUM.ph,x.params.MEDIUM.temperature_Celsius,x.params.Vial,x.params.guidance,x.reference_year
0,JRCNM01001a,,None,,NM-101 (TiO2 6 nm),NPO_1486,NNRG-a51b2e58-4105-9643-3016-3f4b431171e2,L.L. (LOWER LIMIT),32.32,,...,0,30,"CaCl2.2H2O 294.00 %v/v,MgSO4.7H2O 12325.00 %...",NaN,884,NaN,22,JRCNM01001a990498,SOP-Toxicity Test with Daphnia magna (without ...,2016
1,JRCNM01001a,,None,,NM-101 (TiO2 6 nm),NPO_1486,NNRG-a51b2e58-4105-9643-3016-3f4b431171e2,U.L. (UPPER LIMIT),106.03,,...,0,30,"CaCl2.2H2O 294.00 %v/v,MgSO4.7H2O 12325.00 %...",NaN,884,NaN,22,JRCNM01001a990498,SOP-Toxicity Test with Daphnia magna (without ...,2016
2,JRCNM01001a,,None,,NM-101 (TiO2 6 nm),NPO_1486,NNRG-a51b2e58-4105-9643-3016-3f4b431171e2,LC20 LETHAL CONCENTRATION,61.20,,...,0,30,"CaCl2.2H2O 294.00 %v/v,MgSO4.7H2O 12325.00 %...",NaN,884,NaN,22,JRCNM01001a990498,SOP-Toxicity Test with Daphnia magna (without ...,2016
3,JRCNM01001a,,None,,NM-101 (TiO2 6 nm),NPO_1486,NNRG-a51b2e58-4105-9643-3016-3f4b431171e2,L.L. (LOWER LIMIT),89.32,,...,0,30,"CaCl2.2H2O 294.00 %v/v,MgSO4.7H2O 12325.00 %...",NaN,884,NaN,22,JRCNM01001a990498,SOP-Toxicity Test with Daphnia magna (without ...,2016
4,JRCNM01001a,,None,,NM-101 (TiO2 6 nm),NPO_1486,NNRG-a51b2e58-4105-9643-3016-3f4b431171e2,U.L. (UPPER LIMIT),281.47,,...,0,30,"CaCl2.2H2O 294.00 %v/v,MgSO4.7H2O 12325.00 %...",NaN,884,NaN,22,JRCNM01001a990498,SOP-Toxicity Test with Daphnia magna (without ...,2016


In [14]:
import numpy as np
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: red' if v else '' for v in is_max]

for criteria in ["value.lo"]:
    tmp = pd.pivot_table(df, values=criteria, index=['public.name'], columns=['x.oht.top','x.oht.section','x.guidance','value.endpoint','value.qualifier.lo','value.unit'], aggfunc=np.mean).reset_index()
    #tmp.style.highlight_null(null_color='red')
    
    display(tmp.style.apply(highlight_max,subset=top_sections))

In [15]:
tmp=df.groupby(by=["public.name","x.guidance","value.endpoint","value.qualifier.lo","value.unit"]).agg({"value.lo" : ["min","max","mean","std","count"]}).reset_index()
tmp.columns = ["_".join(x) for x in tmp.columns.ravel()]
print("Substances {}".format(tmp.shape[0]))
display(tmp)


Substances 14


,public.name_,x.guidance_,value.endpoint_,value.qualifier.lo_,value.unit_,value.lo_min,value.lo_max,value.lo_mean,value.lo_std,value.lo_count
0,JRCNM01001a,SOP-TOXICITY TEST WITH DAPHNIA MAGNA (WITHOUT ...,L.L. (LOWER LIMIT),,mg/l,32.320,89.320,60.820,40.305087,2
1,JRCNM01001a,SOP-TOXICITY TEST WITH DAPHNIA MAGNA (WITHOUT ...,LC20 LETHAL CONCENTRATION,,,61.200,61.200,61.200,NaN,1
2,JRCNM01001a,SOP-TOXICITY TEST WITH DAPHNIA MAGNA (WITHOUT ...,LC50 LETHAL CONCENTRATION,,,131.140,131.140,131.140,NaN,1
3,JRCNM01001a,SOP-TOXICITY TEST WITH DAPHNIA MAGNA (WITHOUT ...,U.L. (UPPER LIMIT),,mg/l,106.030,281.470,193.750,124.054814,2
4,JRCNM01001a,SOP-TOXICITY TEST WITH MICROALGAE P.SUBCAPITAT...,EC10 LETHAL CONCENTRATION,,mg/l,0.093,0.093,0.093,0.000000,3
5,JRCNM01001a,SOP-TOXICITY TEST WITH MICROALGAE P.SUBCAPITAT...,EC50 LETHAL CONCENTRATION,,mg/l,43.090,43.090,43.090,0.000000,3
6,JRCNM01001a,SOP-TOXICITY TEST WITH MICROALGAE P.SUBCAPITAT...,L.L. (LOWER LIMIT),,mg/l,1.480,19.240,10.360,12.558216,2
7,JRCNM01001a,SOP-TOXICITY TEST WITH MICROALGAE P.SUBCAPITAT...,LC10 LETHAL CONCENTRATION,,,4.380,4.380,4.380,NaN,1
8,JRCNM01001a,SOP-TOXICITY TEST WITH MICROALGAE P.SUBCAPITAT...,LC50 LETHAL CONCENTRATION,,,50.570,50.570,50.570,NaN,1
9,JRCNM01001a,SOP-TOXICITY TEST WITH MICROALGAE P.SUBCAPITAT...,U.L. (UPPER LIMIT),,mg/l,12.980,132.930,72.955,84.817458,2


In [20]:
#Communication with Jaqpot
#pip install git+https://github.com/KinkyDesign/jaqpotpy.git


jaqpot = Jaqpot("http://api.jaqpot.org/jaqpot/services/")
jaqpot.request_key("guest", "guest")
jaqpot.upload_dataset(df=tmp, title="Nanoreg Substances", description="Descriptors derived from Nanoreg Substances")

Dataset created with id: 776b879ece11405d8985ecc34f8eb859
